<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

We use the example from [3]: 10 students are asked to rate three
teaching methods - tutorial, lecture, and seminar - on a scale of 1-5,
with 1 being the lowest and 5 being the highest. We have decided that
a confidence level of 99% is required to reject the null hypothesis in
favor of our alternative: that the seminar will have the highest ratings
and the tutorial will have the lowest. Initially, the data have been
tabulated with each row representing an individual student's ratings of
the three methods in the following order: tutorial, lecture, seminar.


In [ ]:
table = [[3, 4, 3],
         [2, 2, 4],
         [3, 3, 5],
         [1, 3, 2],
         [2, 3, 2],
         [2, 4, 5],
         [1, 2, 4],
         [3, 4, 4],
         [2, 4, 5],
         [1, 3, 4]]

Because the tutorial is hypothesized to have the lowest ratings, the
column corresponding with tutorial rankings should be first; the seminar
is hypothesized to have the highest ratings, so its column should be last.
Since the columns are already arranged in this order of increasing
predicted mean, we can pass the table directly into `page_trend_test`.


In [ ]:
from scipy.stats import page_trend_test
res = page_trend_test(table)
res

PageTrendTestResult(statistic=133.5, pvalue=0.0018191161948127822,
                    method='exact')

This *p*-value indicates that there is a 0.1819% chance that
the $L$ statistic would reach such an extreme value under the null
hypothesis. Because 0.1819% is less than 1%, we have evidence to reject
the null hypothesis in favor of our alternative at a 99% confidence level.

The value of the $L$ statistic is 133.5. To check this manually,
we rank the data such that high scores correspond with high ranks, settling
ties with an average rank:


In [ ]:
from scipy.stats import rankdata
ranks = rankdata(table, axis=1)
ranks

array([[1.5, 3. , 1.5],
       [1.5, 1.5, 3. ],
       [1.5, 1.5, 3. ],
       [1. , 3. , 2. ],
       [1.5, 3. , 1.5],
       [1. , 2. , 3. ],
       [1. , 2. , 3. ],
       [1. , 2.5, 2.5],
       [1. , 2. , 3. ],
       [1. , 2. , 3. ]])

We add the ranks within each column, multiply the sums by the
predicted ranks, and sum the products.


In [ ]:
import numpy as np
m, n = ranks.shape
predicted_ranks = np.arange(1, n+1)
L = (predicted_ranks * np.sum(ranks, axis=0)).sum()
res.statistic == L

True

As presented in [3], the asymptotic approximation of the *p*-value is the
survival function of the normal distribution evaluated at the standardized
test statistic:


In [ ]:
from scipy.stats import norm
E0 = (m*n*(n+1)**2)/4
V0 = (m*n**2*(n+1)*(n**2-1))/144
Lambda = (L-E0)/np.sqrt(V0)
p = norm.sf(Lambda)
p

0.0012693433690751756

This does not precisely match the *p*-value reported by `page_trend_test`
above. The asymptotic distribution is not very accurate, nor conservative,
for $m \leq 12$ and $n \leq 8$, so `page_trend_test` chose to
use ``method='exact'`` based on the dimensions of the table and the
recommendations in Page's original paper [1]. To override
`page_trend_test`'s choice, provide the `method` argument.


In [ ]:
res = page_trend_test(table, method="asymptotic")
res

PageTrendTestResult(statistic=133.5, pvalue=0.0012693433690751756,
                    method='asymptotic')

If the data are already ranked, we can pass in the ``ranks`` instead of
the ``table`` to save computation time.


In [ ]:
res = page_trend_test(ranks,             # ranks of data
                      ranked=True,       # data is already ranked
                      )
res

PageTrendTestResult(statistic=133.5, pvalue=0.0018191161948127822,
                    method='exact')

Suppose the raw data had been tabulated in an order different from the
order of predicted means, say lecture, seminar, tutorial.


In [ ]:
table = np.asarray(table)[:, [1, 2, 0]]

Since the arrangement of this table is not consistent with the assumed
ordering, we can either rearrange the table or provide the
`predicted_ranks`. Remembering that the lecture is predicted
to have the middle rank, the seminar the highest, and tutorial the lowest,
we pass:


In [ ]:
res = page_trend_test(table,             # data as originally tabulated
                      predicted_ranks=[2, 3, 1],  # our predicted order
                      )
res

PageTrendTestResult(statistic=133.5, pvalue=0.0018191161948127822,
                    method='exact')